This is necessary for having figures directly in the notebook.

In [1]:
%matplotlib inline

Import necessary modules

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot
import seaborn as sns

from tridesclous import SpikeSorter, SpikeSortingWindow

We download Locust dataset from zenedo.
We take 3 trials in this file.

In [3]:
from urllib.request import urlretrieve
import os
import h5py

name = 'locust20010201.hdf5'
distantfile = 'https://zenodo.org/record/21589/files/'+name
localfile = name
if not os.path.exists(localfile):
    urlretrieve(distantfile, localfile)
hdf = h5py.File(localfile,'r')

# read 3 trials (=3 segments)
ch_names = ['ch09','ch11','ch13','ch16']
trial_names = ['trial_01', 'trial_02', 'trial_03']
sigs_by_trials = []
for trial_name in trial_names:
    sigs = np.array([hdf['Continuous_1'][trial_name][name][...] for name in ch_names]).transpose()
    sigs = (sigs.astype('float32') - 2**15.) / 2**15
    sigs_by_trials.append(sigs)

sampling_rate = 15000.

# SpikeSorter: insert several segments

Create our data manager and append some data into it.
Note that our data is already filtered.

In [4]:
spikesorter = SpikeSorter(dirname = 'Dataset several segment')

for seg_num in range(3):
    sigs = sigs_by_trials[seg_num]
    spikesorter.dataio.append_signals(sigs, seg_num = seg_num,t_start = 0.+5*seg_num, 
                sampling_rate =  sampling_rate, already_hp_filtered = True, channels = ch_names)

AssertionError: data already in store for seg_num 0

In [5]:
print(spikesorter.summary(level=1))

DataIO <139896538925712>
Workdir: Dataset several segment/data.h5
sampling_rate: 15000.0
nb_channel: 4
nb_segments: 3
Segment 0
    duration : 28.7698s.
    times range : 0.0 - 28.7698
    nb_peaks : 1078
Segment 1
    duration : 28.769800000000004s.
    times range : 5.0 - 33.769800000000004
    nb_peaks : 1140
Segment 2
    duration : 28.769800000000004s.
    times range : 10.0 - 38.769800000000004
    nb_peaks : 899
Peak Cluster



# Detect peak an dextract waveform in one function

In [6]:
spikesorter.detect_peaks_extract_waveforms(seg_nums = 'all',  threshold=-4, 
                                    peak_sign = '-', n_span = 2, n_left=-30, n_right=50)
print(spikesorter.summary(level=1))

DataIO <139896538925712>
Workdir: Dataset several segment/data.h5
sampling_rate: 15000.0
nb_channel: 4
nb_segments: 3
Segment 0
    duration : 28.7698s.
    times range : 0.0 - 28.7698
    nb_peaks : 1078
Segment 1
    duration : 28.769800000000004s.
    times range : 5.0 - 33.769800000000004
    nb_peaks : 1140
Segment 2
    duration : 28.769800000000004s.
    times range : 10.0 - 38.769800000000004
    nb_peaks : 899
Peak Cluster



# project and find cluster

In [7]:
spikesorter.project(method = 'pca', n_components = 5)
spikesorter.find_clusters(7)
spikesorter.refresh_colors(reset=True, palette = 'husl')
print(spikesorter.summary(level=1))
print(spikesorter.cluster_count)

DataIO <139896538925712>
Workdir: Dataset several segment/data.h5
sampling_rate: 15000.0
nb_channel: 4
nb_segments: 3
Segment 0
    duration : 28.7698s.
    times range : 0.0 - 28.7698
    nb_peaks : 1078
Segment 1
    duration : 28.769800000000004s.
    times range : 5.0 - 33.769800000000004
    nb_peaks : 1140
Segment 2
    duration : 28.769800000000004s.
    times range : 10.0 - 38.769800000000004
    nb_peaks : 899
Peak Cluster
  #0: 255
  #1: 1303
  #2: 384
  #3: 116
  #4: 210
  #5: 206
  #6: 642

label
0     255
1    1303
2     384
3     116
4     210
5     206
6     642
Name: label, dtype: int64


# Open interactive windows

In [10]:
%gui qt4
import pyqtgraph as pg
app = pg.mkQApp()
win = SpikeSortingWindow(spikesorter)
win.show()